In [ ]:
import os
import setGPU
# edit depending on where Vivado is installed:
# os.environ['PATH'] = '/<Xilinx installation directory>/Vivado/<version>/bin:' + os.environ['PATH']
os.environ['PATH'] = '/xilinx/Vivado/2019.2/bin:' + os.environ['PATH']
import tensorflow as tf
from qkeras.utils import _add_supported_quantized_objects
import hls4ml
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model

In [ ]:
def yaml_load(config):
    with open(config) as stream:
        param = yaml.safe_load(stream)
        

def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

In [ ]:
model_file_path = 'model_QInteractionNetwork_nconst_8_nbits_8.h5'

In [ ]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)    

model = load_model(model_file_path, custom_objects=co)
model.summary()

In [ ]:
import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model'] = {}
config['Model']['ReuseFactor'] = 1
config['Model']['Strategy'] = 'Latency'
config['Model']['Precision'] = 'ap_fixed<16,6>'

cfg = hls4ml.converters.create_vivado_config(fpga_part='xc7z020clg400-1')
cfg['HLSConfig'] = config
cfg['IOType'] = 'io_parallel'
cfg['Backend'] = 'Vivado'
cfg['ClockPeriod'] = 10
cfg['KerasModel'] = model
cfg['OutputDir'] = 'hls_output'

print("-----------------------------------")
print_dict(cfg)
print("-----------------------------------")

In [ ]:
# just test transpose
from tensorflow.keras.layers import Input, Permute, Flatten, Reshape
inputs = Input(shape=(8,3,))
outputs = Permute((2,1))(inputs)

model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model'] = {}
config['Model']['ReuseFactor'] = 1
config['Model']['Strategy'] = 'Latency'
config['Model']['Precision'] = 'ap_fixed<16,6>'

cfg = hls4ml.converters.create_vivado_config(fpga_part='xc7z020clg400-1')
cfg['HLSConfig'] = config
cfg['IOType'] = 'io_parallel'
cfg['Backend'] = 'Vivado'
cfg['ClockPeriod'] = 10
cfg['KerasModel'] = model
cfg['OutputDir'] = 'hls_output'

print("-----------------------------------")
print_dict(cfg)
print("-----------------------------------")

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

In [ ]:
x = np.random.rand(1, 3, 8)
print(x.flatten())
print(x.transpose().flatten())

In [ ]:
print(hls_model.predict(x))